In [1]:
n_qubits = 4
depth = 1
ansatz_id = 11
subgroup_size = 2

import json
from ansatz import Ansatz
from utils import find_original_param, get_subgroup_unitaries, pennylane_to_qiskit

from twirler.symmetry_groups import create_symmetric_group
from twirler.generators import get_ansatz_generators
from twirler.twirling import apply_twirling_to_generators

from qiskit import QuantumCircuit, transpile
from qiskit.circuit.library import PauliEvolutionGate, UnitaryGate
from qiskit.quantum_info import SparsePauliOp

    
S = create_symmetric_group(n_qubits)

with open(f"groups/subgroups_{n_qubits}.json", "r") as f:
    subgroups = json.load(f)

subgroup_unitaries = get_subgroup_unitaries(subgroups, S)

super_ansatz = Ansatz(ansatz_id, n_qubits, depth)
circuit, params = super_ansatz.get_QNode()
qiskit_circuit = pennylane_to_qiskit(circuit, n_qubits, params=params)
qiskit_circuit = qiskit_circuit.remove_final_measurements(inplace=False)
print(qiskit_circuit.draw("latex_source"))
params = qiskit_circuit.parameters
ansatz_generators = get_ansatz_generators(super_ansatz.get_ansatz())

for elem in subgroup_unitaries[str(subgroup_size)]:
    unitaries = elem["unitaries"]
    twirled_generators = apply_twirling_to_generators(unitaries, ansatz_generators, n_qubits)
    twirled_circuit = QuantumCircuit(n_qubits)
    for i, (gen_matrix, op_wires, op_name, theta, is_parametric) in enumerate(ansatz_generators):
        twirled_elem = twirled_generators[i]
        if twirled_elem["gate_name"] == op_name and twirled_elem["wires"] == op_wires:
            H = twirled_elem['averaged']
            if is_parametric:
                param = find_original_param(qiskit_circuit, ansatz_generators, i, op_name, op_wires, theta)
            else:
                param = 1.0
            pauli_op = SparsePauliOp.from_operator(H)
            evo_gate = PauliEvolutionGate(pauli_op, time=param)
            twirled_circuit.append(evo_gate, range(n_qubits))
        else:
            raise ValueError(f"Twirled generator for {op_name} on wires {op_wires} not found when {twirled_elem['gate_name']} and {twirled_elem['wires']}")

    twirled_circuit.remove_final_measurements(inplace=True)
    transpiled_circuit = transpile(
            twirled_circuit,
            basis_gates=['rz', 'sx', 'cx', 'h', 'x', 'y', 'z', 'rx', 'ry', 'cz', 'rxx', 'rzz'],
            optimization_level=3
        )
    fig = transpiled_circuit.draw("mpl")
    fig.savefig(f"twirled_circuit_n{n_qubits}_d{depth}_a{ansatz_id}_k{subgroup_size}.png")
    print(transpiled_circuit.draw("latex_source"))
    break


\documentclass[border=2px]{standalone}

\usepackage[braket, qm]{qcircuit}
\usepackage{graphicx}

\begin{document}
\scalebox{1.0}{
\Qcircuit @C=1.0em @R=0.2em @!R { \\
	 	\nghost{{q}_{0} :  } & \lstick{{q}_{0} :  } & \gate{\mathrm{R_Y}\,(\mathrm{x0})} & \gate{\mathrm{R_Z}\,(\mathrm{x1})} & \targ & \qw & \qw & \qw & \qw & \qw\\
	 	\nghost{{q}_{1} :  } & \lstick{{q}_{1} :  } & \gate{\mathrm{R_Y}\,(\mathrm{x2})} & \gate{\mathrm{R_Z}\,(\mathrm{x3})} & \ctrl{-1} & \gate{\mathrm{R_Y}\,(\mathrm{x8})} & \gate{\mathrm{R_Z}\,(\mathrm{x9})} & \targ & \qw & \qw\\
	 	\nghost{{q}_{2} :  } & \lstick{{q}_{2} :  } & \gate{\mathrm{R_Y}\,(\mathrm{x4})} & \gate{\mathrm{R_Z}\,(\mathrm{x5})} & \targ & \gate{\mathrm{R_Y}\,(\mathrm{x10})} & \gate{\mathrm{R_Z}\,(\mathrm{x11})} & \ctrl{-1} & \qw & \qw\\
	 	\nghost{{q}_{3} :  } & \lstick{{q}_{3} :  } & \gate{\mathrm{R_Y}\,(\mathrm{x6})} & \gate{\mathrm{R_Z}\,(\mathrm{x7})} & \ctrl{-1} & \qw & \qw & \qw & \qw & \qw\\
\\ }}
\end{document}
\documentclass[border=2px]